In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns",999)

In [2]:
def all_values(df):
    for i in df.columns:
        print("")
        print(i,":", df[i].value_counts(),"\n")

### Ideas
* Instead of religion, just do "seriousness" of religion
* Instead of horoscope sign, just do "seriousness" of horoscopes

### Simple, basic transformations
* **Drop** essays, last_online, drugs, drinks, diet, religion, speaks, location, ethnicity, offspring, sign
*

In [24]:
cupid = pd.read_csv('okcupid/profiles.csv')

In [16]:
cupid = cupid.drop(['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8', 'essay9',
                    'last_online', 'drinks', 'drugs', 'diet', 'religion', 'speaks', 'location', 'ethnicity', 'offspring', 'sign'], axis=1)

In [17]:
cupid.shape

(59946, 11)

In [18]:
cupid.columns

Index(['age', 'body_type', 'education', 'height', 'income', 'job',
       'orientation', 'pets', 'sex', 'smokes', 'status'],
      dtype='object')

In [19]:
cupid.sample()

,age,body_type,education,height,income,job,orientation,pets,sex,smokes,status
21171,32,skinny,working on ph.d program,74.0,20000,science / tech / engineering,straight,NaN,m,no,seeing someone


In [25]:
for i in cupid.columns:
    print(i,": ",len(cupid[i].unique()))

age :  54
body_type :  13
diet :  19
drinks :  7
drugs :  4
education :  33
essay0 :  54351
essay1 :  51517
essay2 :  48636
essay3 :  43534
essay4 :  49261
essay5 :  48964
essay6 :  43604
essay7 :  45555
essay8 :  39325
essay9 :  45444
ethnicity :  218
height :  61
income :  13
job :  22
last_online :  30123
location :  199
offspring :  16
orientation :  3
pets :  16
religion :  46
sex :  2
sign :  49
smokes :  6
speaks :  7648
status :  5


In [26]:
cupid.isna().sum()

age                0
body_type       5296
diet           24395
drinks          2985
drugs          14080
education       6628
essay0          5488
essay1          7572
essay2          9638
essay3         11476
essay4         10537
essay5         10850
essay6         13771
essay7         12451
essay8         19225
essay9         12603
ethnicity       5680
height             3
income             0
job             8198
last_online        0
location           0
offspring      35561
orientation        0
pets           19921
religion       20226
sex                0
sign           11056
smokes          5512
speaks            50
status             0
dtype: int64

In [29]:
# print('smokes:',cupid.smokes.unique())
# print('drinks:',cupid.drinks.unique())
# print('drugs:',cupid.drugs.unique())
print('last_online:',cupid.last_online.unique())

last_online: ['2012-06-28-20-30' '2012-06-29-21-41' '2012-06-27-09-10' ...
 '2012-06-02-08-16' '2012-02-17-20-44' '2012-06-14-16-51']


In [35]:
cupid.smokes.value_counts()

no                43896
sometimes          3787
when drinking      3040
yes                2231
trying to quit     1480
Name: smokes, dtype: int64

In [104]:
cupid.ethnicity.value_counts()

white                                                                                                      32831
asian                                                                                                       6134
hispanic / latin                                                                                            2823
black                                                                                                       2008
other                                                                                                       1706
hispanic / latin, white                                                                                     1301
indian                                                                                                      1077
asian, white                                                                                                 811
white, other                                                                                    

In [71]:
cupid.dropna().shape

(7127, 20)

In [73]:
cupid.dropna().smokes.value_counts()

no                5737
sometimes          474
when drinking      378
yes                305
trying to quit     233
Name: smokes, dtype: int64

In [33]:
for i in cupid.columns:
    print(i,": ",len(cupid[i].unique()))

age :  54
body_type :  13
diet :  19
drinks :  7
drugs :  4
education :  33
ethnicity :  218
height :  61
income :  13
job :  22
last_online :  30123
location :  199
offspring :  16
orientation :  3
pets :  16
religion :  46
sex :  2
sign :  49
smokes :  6
speaks :  7648
status :  5


In [103]:
cupid.speaks.value_counts()

english                                                                                        21828
english (fluently)                                                                              6628
english (fluently), spanish (poorly)                                                            2059
english (fluently), spanish (okay)                                                              1917
english (fluently), spanish (fluently)                                                          1288
english, spanish                                                                                 859
english (fluently), french (poorly)                                                              756
english, spanish (okay)                                                                          655
english, spanish (poorly)                                                                        609
english (fluently), chinese (fluently)                                                     